In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST/data', one_hot=True)

Extracting MNIST/data\train-images-idx3-ubyte.gz
Extracting MNIST/data\train-labels-idx1-ubyte.gz
Extracting MNIST/data\t10k-images-idx3-ubyte.gz
Extracting MNIST/data\t10k-labels-idx1-ubyte.gz


In [4]:
(train_data, validation_data, test_data) = (mnist.train, mnist.validation, mnist.test)

In [6]:
train_data.num_examples

55000

In [32]:
with tf.name_scope('Inputs') as scope:
    #x = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='images')
    x = tf.constant(train_data.images[:5], name='images')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='2d_images')
    #y_correct = tf.placeholder(shape=[None, 10], dtype=tf.float32, name='correct_output')
    y_correct = tf.constant(train_data.labels[:5], name='labels')

In [7]:
sess = tf.Session()

In [33]:
sess.run(tf.shape(x_image))

array([ 5, 28, 28,  1])

In [34]:
# first convolution
filter_conv1 = 8
with tf.name_scope('First_convolution') as scope:
    W_conv1 = tf.truncated_normal(shape=[6, 6, 1, filter_conv1], stddev=0.1)
    b_conv1 = tf.zeros([filter_conv1])
    h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 2, 2, 1], padding='SAME') + b_conv1   # (None, 14, 14, 8)
    h_conv1_relu = tf.nn.relu(h_conv1)

In [35]:
sess.run(tf.shape(h_conv1))

array([ 5, 14, 14,  8])

In [36]:
with tf.name_scope('First_pooling') as scope:
    h_pool_1 = tf.nn.max_pool(h_conv1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 7, 7, 8)

In [37]:
sess.run(tf.shape(h_pool_1))

array([5, 7, 7, 8])

In [38]:
# second convolution
filter_conv2 = 16
with tf.name_scope('Second_convolution') as scope:
    W_conv2 = tf.truncated_normal(shape=[4, 4, filter_conv1, filter_conv2], stddev=0.1)
    b_conv2 = tf.zeros([filter_conv2])
    h_conv2 = tf.nn.conv2d(h_pool_1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2   # (None, 7, 7, 16)
    h_conv2_relu = tf.nn.relu(h_conv2)

In [39]:
# second convolution
with tf.name_scope('Second_pooling') as scope:
    h_pool_2 = tf.nn.max_pool(h_conv2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 7, 7, 16)

In [40]:
sess.run(tf.shape(h_pool_2))

array([ 5,  4,  4, 16])

In [41]:
# flatten
fc_input = tf.reshape(h_pool_2, [-1, 4*4*16])

In [42]:
sess.run(tf.shape(fc_input))

array([  5, 256])

In [43]:
# Fully Connected
neuron_1 = 4*4*16
#neuron_2 = 256

In [44]:
with tf.name_scope('First_hidden') as scope:
    W_1 = tf.truncated_normal([neuron_1, 10])
    b_1 = tf.Variable(0.0, [10])
    y = tf.matmul(fc_input, W_1) + b_1

In [46]:
sess.run(tf.shape(y))

array([ 5, 10])

In [51]:
with tf.name_scope('Cost') as scope:
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_correct, logits=y))

In [52]:
# Optimization
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluate model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())
file_writer = tf.summary.FileWriter("CNN_baitap_1", sess.graph)
# create a summary for our cost and accuracy
tf.summary.scalar("cost_summary", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
# merge all summaries into a single operation which we can execute in a session 
summary_step = tf.summary.merge_all()
validation_size = 200
for i in range(501):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    if i%50 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_xs, y_correct: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        valid_xs, valid_ys = mnist.validation.next_batch(validation_size)
        print("Accuracy validation: {}".format(accuracy.eval(
                feed_dict = {x: valid_xs, y_correct: valid_ys,keep_prob: 1.0 })))
    _, summary = sess.run([train_step, summary_step], 
            feed_dict={x: batch_xs, y_correct: batch_ys, keep_prob: 0.6})
    # logging
    file_writer.add_summary(summary, i)
    
print("Accuracy: {}".format(accuracy.eval(feed_dict = {x: mnist.test.images[:validation_size], 
                                               y_correct: mnist.test.labels[:validation_size], 
                                               keep_prob: 1.0})))